In [1]:
from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .appName("iceberg") \
    .master("local[*]") \
    .config("spark.jars.ivy","/home/brijeshdhaker/.ivy2") \
    .config("spark.jars.packages","org.apache.iceberg:iceberg-spark-runtime-3.4_2.12:1.6.1,org.apache.hadoop:hadoop-aws:3.0.0") \
    .config("spark.sql.extensions","org.apache.iceberg.spark.extensions.IcebergSparkSessionExtensions") \
    .config("spark.eventLog.enabled", "true") \
    .config("spark.eventLog.dir", "/apps/var/logs/spark-events") \
    .config("spark.history.fs.logDirectory", "/apps/var/logs/spark-events") \
    .config("spark.hadoop.fs.s3a.endpoint", "http://minio.sandbox.net:9010") \
    .config("spark.hadoop.fs.s3a.access.key", "pgm2H2bR7a5kMc5XCYdO") \
    .config("spark.hadoop.fs.s3a.secret.key", "zjd8T0hXFGtfemVQ6AH3yBAPASJNXNbVSx5iddqG") \
    .config("spark.hadoop.fs.s3a.path.style.access", "true") \
    .config("spark.hadoop.fs.s3a.aws.credentials.provider", "org.apache.hadoop.fs.s3a.SimpleAWSCredentialsProvider") \
    .config("spark.hadoop.fs.s3a.impl", "org.apache.hadoop.fs.s3a.S3AFileSystem") \
    .getOrCreate()

spark

:: loading settings :: url = jar:file:/opt/spark-3.4.1/jars/ivy-2.5.1.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /home/brijeshdhaker/.ivy2/cache
The jars for the packages stored in: /home/brijeshdhaker/.ivy2/jars
org.apache.iceberg#iceberg-spark-runtime-3.4_2.12 added as a dependency
org.apache.hadoop#hadoop-aws added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-05e4aeeb-7d1e-437c-97fa-e498a9d55e2c;1.0
	confs: [default]
	found org.apache.iceberg#iceberg-spark-runtime-3.4_2.12;1.6.1 in central
	found org.apache.hadoop#hadoop-aws;3.0.0 in central
	found com.amazonaws#aws-java-sdk-bundle;1.11.199 in central
downloading https://repo1.maven.org/maven2/org/apache/iceberg/iceberg-spark-runtime-3.4_2.12/1.6.1/iceberg-spark-runtime-3.4_2.12-1.6.1.jar ...
	[SUCCESSFUL ] org.apache.iceberg#iceberg-spark-runtime-3.4_2.12;1.6.1!iceberg-spark-runtime-3.4_2.12.jar (8555ms)
:: resolution report :: resolve 12318ms :: artifacts dl 8730ms
	:: modules in use:
	com.amazonaws#aws-java-sdk-bundle;1.11.199 from central in [default]
	org.apache.hadoop#hadoop-

In [2]:
from datetime import datetime, date
import pandas as pd
from pyspark.sql import Row

df = spark.createDataFrame([
    Row(a=1, b=2., c='string1', d=date(2000, 1, 1), e=datetime(2000, 1, 1, 12, 0)),
    Row(a=2, b=3., c='string2', d=date(2000, 2, 1), e=datetime(2000, 1, 2, 12, 0)),
    Row(a=4, b=5., c='string3', d=date(2000, 3, 1), e=datetime(2000, 1, 3, 12, 0))
])
df

DataFrame[a: bigint, b: double, c: string, d: date, e: timestamp]

In [3]:
# All DataFrames above result same.
df.show()
df.printSchema()

+---+---+-------+----------+-------------------+
|  a|  b|      c|         d|                  e|
+---+---+-------+----------+-------------------+
|  1|2.0|string1|2000-01-01|2000-01-01 12:00:00|
|  2|3.0|string2|2000-02-01|2000-01-02 12:00:00|
|  4|5.0|string3|2000-03-01|2000-01-03 12:00:00|
+---+---+-------+----------+-------------------+

root
 |-- a: long (nullable = true)
 |-- b: double (nullable = true)
 |-- c: string (nullable = true)
 |-- d: date (nullable = true)
 |-- e: timestamp (nullable = true)



In [2]:
# spark is an existing SparkSession
df = spark.read.csv("s3a://datasets/flight-data/airlines.dat")
# Displays the content of the DataFrame to stdout
df.show()
# +----+-------+
# | age|   name|
# +----+-------+
# |null|Michael|
# |  30|   Andy|
# |  19| Justin|
# +----+-------+

+---+--------------------+---+----+----+--------------+--------------+---+
|_c0|                 _c1|_c2| _c3| _c4|           _c5|           _c6|_c7|
+---+--------------------+---+----+----+--------------+--------------+---+
| -1|             Unknown| \N|   -| N/A|            \N|            \N|  Y|
|  1|      Private flight| \N|   -| N/A|          null|          null|  Y|
|  2|         135 Airways| \N|null| GNL|       GENERAL| United States|  N|
|  3|       1Time Airline| \N|  1T| RNX|       NEXTIME|  South Africa|  Y|
|  4|2 Sqn No 1 Elemen...| \N|null| WYT|          null|United Kingdom|  N|
|  5|     213 Flight Unit| \N|null| TFU|          null|        Russia|  N|
|  6|223 Flight Unit S...| \N|null| CHD|CHKALOVSK-AVIA|        Russia|  N|
|  7|   224th Flight Unit| \N|null| TTF|    CARGO UNIT|        Russia|  N|
|  8|         247 Jet Ltd| \N|null| TWF|  CLOUD RUNNER|United Kingdom|  N|
|  9|         3D Aviation| \N|null| SEC|       SECUREX| United States|  N|
| 10|         40-Mile Air